# Problem Set 5: Optimisation

## Question 1: Rosenbrock Constrained

In [1]:
import matplotlib.pyplot as plt
import autograd.numpy as np
import numdifftools as nd
from scipy.optimize import minimize

### 1(a) Sequential Quadratic Programming with Newton's Method

In [2]:
"""Adding a, b into arguments"""
def rosen(X, a, b):     # Objective function
    f = []
    for i in range(len(X)-1):   # Iterate over N-1 variables in X
        f_i = ((a*(1-X[i])**2) + b*((X[i+1] - X[i]**2))**2)
        f.append(f_i)
    return sum(f)

In [3]:
##############
### PART A ###
##############

"""
Throughout part A, we use the following arguments for testing purposes:
X = [1,2,3]
lambda = 1
a = 1
b = 1
r = 1
"""
Y = [ 1,  2,   3,   1]
#     ^   ^    ^    ^  
#   X[0] X[1] X[2] lam
a = 1
b = 1
r = 1

#### i)

<h2><center>$\mathcal{L}$ = f(x) -  $\lambda$$\cdot$g(x)</center></h2>

<h2><center>f(x) = $\sum_{i=1}^{n-1}{(a(1-x_i}^2) + b(x_{i+1}-x^2_i)^2)$</center></h2>
<h2><center>g(x) = $\sum_{i=1}^{n}{x^2_i}  - r$</center></h2>


<h2><center>$\mathcal{L}$ = $\sum_{i=1}^{n-1}{(a(1-x_i}^2) + b(x_{i+1}-x^2_i)^2)$ - $\lambda$$\cdot$$(\sum_{i=1}^{n}{x^2_i}  - r)$</center></h2>

#### ii)

In [4]:
# ------ ii
def constraint(X, r):
    g = []
    for i in range(0, len(X)):  # Iterate over N variables in X
        g_i=(X[i])**2
        g.append(g_i)
    return sum(g) - r           # Minus r at the end to complete constraint expression

"""To be able to use the gradient functions in later questions, we combine the x variables and lambda into one argument vector, where the last variable is lambda"""
def lagrangian(Y, a, b, r):
    X = Y[0:-1]
    lam = Y[-1]
    L = rosen(X, a, b) - lam*(constraint(X, r))
    return L 

# EXAMPLE:
lagrangian(Y,a,b,r)     # ANSWER: -10

-10

#### iii)

In [5]:
# ------ iii
# From numdifftools, we can use the Hessian function
def gradient(Y,a,b,r):
    return nd.core.Jacobian(lagrangian)(Y, a, b, r)

gradient(Y, a, b, r)    # ANSWER: [[-6, 8, -8, -13]]

array([[ -6.,   8.,  -8., -13.]])

#### iv)

In [6]:
# ------ iv
#Finite Differences (Without - eps because there is no need to minus it as well.)
def fin_diff(Y, a, b, r, eps=1e-6):             # NB: The unpacking of Y is done in the lagrangian function.
    # Finite differences for X variables
    grad_x = []
    for i in range(len(Y)-1):                   # We iterate until the second last variable because after that is lambda

        X_eps = Y.copy()                        # Reset array so we can perform a partial derivative
        X_eps[i] = X_eps[i] + eps               # Increase ith variable by marginal amount
        y_eps = lagrangian(X_eps, a, b, r)      # Perform lagrangian on new array
        y = lagrangian(Y, a, b, r)              # Perform lagrangian on original array
        grad_x.append((y_eps - y) / (eps))      # Calculate gradient between the two arrays

    # Finite differences for lambda
    lam_eps = Y.copy()
    lam_eps[-1] = lam_eps[-1] + eps             # We use [-1] as this refers to our lambda value
    y_eps = lagrangian(lam_eps, a, b, r)    
    y = lagrangian(Y, a, b, r)
    grad_lam = (y_eps - y) / (eps)

    # Combine gradients into one array
    grad = grad_x.copy()
    grad.append(grad_lam)
    return grad

fin_diff([1,2,3,1], 1, 1, 1)
# ANSWER = ([-5.9999, 8.00001, -8.000000001, -12.9999999]), which closely matches gradient function  

[-5.999998000660867,
 8.000019001030978,
 -8.000000001118224,
 -12.999999999152578]

In [7]:
# Using RMSE to test the gradient function
def RMSE(points, a, b, r):      # Points = array of coordinates
    y = []          # Part iii jacobian gradient function output 
    y_hat = []      # Finite differences function output

    # Calculate gradients
    for point in points:
        # Gradient/Jacobian function
        output_jac = gradient(point, a, b, r)
        y.append(output_jac)

        # Finite differences function
        output_fd = fin_diff(point, a, b, r)
        y_hat.append(output_fd)

    # RMSE Calculation
        # Squared Error Calculation
    squared_error = []
    for k in range(K):
        error = np.linalg.norm(y[k] - y_hat[k])
        squared_error.append(error**2)

        # Sum Squared Errors
    sse = sum(squared_error)

        # RMSE
    RMSE = np.sqrt(sse/K)
    return RMSE

# Create points:
K = 100    # Number of sampling points
N = 6      # Number of variables + Lambda (To use as our argument vector)
input_space = np.random.rand(K, N)

# Find RMSE for different values of b
B = [1, 100, 1000]
for b_ in B:
    output = RMSE(input_space, a, b_, r)
    print(f"b = {b_} : RMSE = {output}")

b = 1 : RMSE = 6.079735380977762e-06
b = 100 : RMSE = 0.000546132799216091
b = 1000 : RMSE = 0.005456641320609471


As b increases, the RMSE increases somewhat linearly

The intution is as follows:
- As b increases, the function becomes steeper (i.e more sensitive to differences in the xi's).
- This makes it more difficult for the finite differences method to approximate the gradient.
- Ideally, a small change in xi should lead to an infinitesimal change in the rosenbrock function.
- Increasing b makes a small change in xi lead to a larger change in the rosenbrock, which is suboptimal when trying to approximate the gradient.


#### Is there a value of b where finite differences stops working well?
- Even with b = 1000, the RMSE for 100 different points is very small.
- We tested up to b = 100000, which produced an RMSE of approximately 0.5.
- So we suggest that any value of b above 100000 makes finite differences not work so reliably.

#### v)

In [8]:
# ------ v
# From numdifftools, we can use the Hessian function
def hessian(Y, a, b, r):
    H = nd.Hessian(lagrangian)
    return H(Y, a, b, r)

hessian(Y,a,b,r)
# ANSWER:
# [[ 4, -4,  0, -2 ],
#  [-4, 38, -8, -4 ],
#  [ 0, -8,  0, -6 ],
#  [-2, -4, -6,  0 ]]

array([[ 4.00000000e+00, -4.00000000e+00,  9.83591475e-17,
        -2.00000000e+00],
       [-4.00000000e+00,  3.80000000e+01, -8.00000000e+00,
        -4.00000000e+00],
       [ 9.83591475e-17, -8.00000000e+00,  0.00000000e+00,
        -6.00000000e+00],
       [-2.00000000e+00, -4.00000000e+00, -6.00000000e+00,
         0.00000000e+00]])

#### vi)

In [9]:
# ------ vi
def search_direction(Y, a, b, r):
    H = hessian(Y,a,b,r)            # Generate Hessian
    J = gradient(Y,a,b,r)           # Generate Jacobian
    J = np.reshape(J, (len(Y),1))   # We need to transform the jacobian into a column vector in order to use linalg.solve

    s_k = np.linalg.solve(H,-J)     # Ensure we input the negative of the Jacobian
    return s_k

search_direction(Y, a, b, r)
# ANSWER
# [[ 0.62903226],
#  [-0.61290323],
#  [-1.96774194],
#  [-0.51612903]]

array([[ 0.62903226],
       [-0.61290323],
       [-1.96774194],
       [-0.51612903]])

#### vii)

In [10]:
# ------ vii
def iterate(Y_k, a, b, r, alpha=1):
    s_k = search_direction(Y_k,a,b,r)               # Generate search direction
    Y_k = np.reshape(np.array(Y_k), (len(Y_k),1))   # Ensuring that Y_k and s_k are the same shape
    s_k = np.reshape(s_k, (len(s_k),1))             # Ensuring that Y_k and s_k are the same shape
    Y_k_1 = Y_k + alpha*s_k             # Adding them together elementwise is easier when they are the same shape
    return Y_k_1

iterate(Y, a, b, r)
# ANSWER:
# [[1.62903226],
#  [1.38709677],
#  [1.03225806],
#  [0.48387097]]

array([[1.62903226],
       [1.38709677],
       [1.03225806],
       [0.48387097]])

#### viii)

In [11]:
# ------ viii
def convergence(Y, a, b, r, tol=1e-6):
    norm = np.linalg.norm(gradient(Y,a,b,r))
    return norm < tol

convergence(Y,a,b,r)
# ANSWER: False (which is expected)

False

#### ix)

In [12]:
# ------ ix
# Set parameters
a = 1
b = 1
r = 2

x_k = []    # Create empty list
def newton(Y_0, a, b, r):
    iter = 0
    y_k = Y_0
    while not convergence(y_k, a, b, r):
        y_k_1 = iterate(y_k, a, b, r)

        y_k = y_k_1.tolist()    # y_k_1 is an np.array but we need to change the input to a list as our functions only take in lists
        y_k = sum(y_k, [])      # This line converts y_k from a list of lists to a single list 
        x_k.append(y_k[0:-1])   # Make sure to not add the value of lambda to the list of x_k
        iter += 1
    print(f"number of iterations: {iter}")

newton([0,0,0], a, b, r)
x_k

number of iterations: 1


[[1.0586533664232292e+18, 0.0]]

Unfortunately our newton function would not work for the points [0,0,0]

After some debugging, we find that the linalg.solve function in our search_direction function returns very high values (up to 1e36).

These numbers are too high when called by functions like the jacobian, which seems to break and automatically outputs a value of [0,0,0], as if the root has been found precisely.

As a result, our convergence checker returns True after 1 itertation and the function ends.

#### x)

In [13]:
# ------ x
def newton_complete(Y_0, a, b, r, tol=1e-6, max_iterations=1000):     # Parameter n not needed as our functions will get n from the length of Y_0
    # Initialize values
    iter = 0 
    y_k = Y_0
    
    # Newton loop:
    while not (convergence(y_k, a, b, r, tol)) or (iter < max_iterations):
        y_k_1 = iterate(y_k, a, b, r)

        y_k = y_k_1.tolist()    # y_k_1 is an np.array but we need to change the input to a list as our functions only take in lists
        y_k = sum(y_k, [])      # This line converts y_k from a list of lists to a single list
        iter += 1

    # Create values to return:
    f_value = rosen(y_k, a, b)
    optimum_x = y_k[0:-1]
    optimum_lam = y_k[-1]
    newton_steps = iter
    norm_DL = np.linalg.norm(gradient(y_k,a,b,r))
    cons_deviation = constraint(y_k[0:-1], r)

    return {"f_value":f_value, "optimum_x":optimum_x, "optimum_lam":optimum_lam, "newton_steps":newton_steps, "norm_DL":norm_DL, "cons_deviation":cons_deviation}

#### xi)

In [14]:
# ------ xi
r = 2

# Create points:
K = 5   # Number of initial points
N = 6   # Number of variables + Lambda (To use as our argument vector)

output = []
for k in range(K):
    Y_0 = np.random.uniform(0, 10, N)
    output.append({f"Initial point {k+1}" : newton_complete(Y_0, a, b, r, max_iterations=100)})
    
output

[{'Initial point 1': {'f_value': 1.3358479582605698,
   'optimum_x': [0.7990113591987212,
    0.7430214763780109,
    0.6799183498147472,
    0.5512916025361428,
    0.20805898219201105],
   'optimum_lam': -0.4607513111180575,
   'newton_steps': 100,
   'norm_DL': 7.043923318186837e-16,
   'cons_deviation': -2.220446049250313e-16}},
 {'Initial point 2': {'f_value': 1.3358479582605693,
   'optimum_x': [0.7990113591987212,
    0.7430214763780109,
    0.6799183498147472,
    0.5512916025361428,
    0.20805898219201102],
   'optimum_lam': -0.4607513111180572,
   'newton_steps': 100,
   'norm_DL': 7.098592150026709e-16,
   'cons_deviation': -2.220446049250313e-16}},
 {'Initial point 3': {'f_value': 1.335847958260569,
   'optimum_x': [0.7990113591987211,
    0.7430214763780107,
    0.6799183498147473,
    0.551291602536143,
    0.20805898219201113],
   'optimum_lam': -0.46075131111805656,
   'newton_steps': 100,
   'norm_DL': 4.749397776179922e-15,
   'cons_deviation': 0.0}},
 {'Initial poin


Overall, the function does not seem to be working as intended. We think it may have to do with our linalg.solve function.

Perhaps the fact that lambda is included in our Jacobian and Hessian may be confusing the linalg.solve function.

### 1(b) Penalty method

In [15]:
# Defining functions:
def rosen_b(X):
    f = []
    for i in range(len(X)-1):
        f_i = (a*(1-X[i])**2) + b*(X[i+1] - X[i]**2)**2
        f.append(f_i)
    return sum(f)

def g(X,r):
    g = []
    for i in range(len(X)):
        g_i = X[i]**2
        g.append(g_i)
    return sum(g)-r

#### i)

In [16]:
# (i)

a = 10
b = 10
r = 10
n = 10
Pk = 1e12
x0= np.zeros(n)

def penalty_method(X, Pk):
    return rosen_b(X) + (Pk*np.linalg.norm(g(X,r)))

res = minimize(penalty_method, x0, args=(Pk), method='L-BFGS-B', options={'maxiter': 10000})

print("Optimal solution:\n", res.x)
print("Objective value (How well L-BFGS minimizes the function):", rosen_b(res.x))
print("Constraint value (How good the optimal solution satisfies the constraint):", g(res.x,r))
print("Number of iterations:", res.nit)

Optimal solution:
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Objective value (How well L-BFGS minimizes the function): 90.0
Constraint value (How good the optimal solution satisfies the constraint): -10.0
Number of iterations: 0


Our minimize function doesn't seem to work $x_0$ = 0. As can be seen from the number of iterations being 0.

We think this is because the penalty function is too high.

We will see in later parts that the iterative Pk method fares much better.

#### ii)

In [17]:
# (ii)

Pk = 1
x0 = np.zeros(n)
for i in range(13):
    print("---------------------------------------------------------------")
    print(f"Testing Penalty Method with Pk = {Pk}")
    res = minimize(penalty_method, x0, args=(Pk), method='L-BFGS-B', options={'maxiter': 10000})

    print("Optimal solution:\n", res.x)
    print("Objective value:", rosen_b(res.x))
    print("Constraint value:", g(res.x,r))
    print("Number of iterations:", res.nit)
    
    x0 = res.x    # Set xk+1 as new solution
    Pk *= 10      # Pk+1 = Pk*10

    print("---------------------------------------------------------------")

---------------------------------------------------------------
Testing Penalty Method with Pk = 1
Optimal solution:
 [0.99999413 0.99999299 0.99998925 0.99999446 1.00000078 1.0000073
 1.00001029 1.00000497 1.00000163 1.00000419]
Objective value: 1.2107985907919064e-08
Constraint value: -3.604421650038603e-09
Number of iterations: 33
---------------------------------------------------------------
---------------------------------------------------------------
Testing Penalty Method with Pk = 10
Optimal solution:
 [0.99999413 0.99999299 0.99998925 0.99999446 1.00000078 1.0000073
 1.00001029 1.00000497 1.00000163 1.00000419]
Objective value: 1.2107985907919064e-08
Constraint value: -3.604421650038603e-09
Number of iterations: 0
---------------------------------------------------------------
---------------------------------------------------------------
Testing Penalty Method with Pk = 100
Optimal solution:
 [0.99999413 0.99999299 0.99998925 0.99999446 1.00000078 1.0000073
 1.00001029 1.

With this method, our results fare much better.

Starting with Pk=1, the minimize function has a bit more room to change values.

We see that combining an iterative increase in Pk with the assignment of $x_{k+1}$ as the previous solution allows us to get much closer to the true optimum values.

However, after the first iteration, the solution is already close enough to the true values, such that the minimize function - which takes in this value as an input - is already satisfied that the optimum solution has been reached. So there isn't much need for many iterations through Pk.

#### iii)

In [18]:
# (iii)

Pk = 1
b = 1000
x0 = np.zeros(n)
for i in range(13):
    print("---------------------------------------------------------------")
    print(f"Testing Penalty Method with Pk = {Pk}")
    res = minimize(penalty_method, x0, args=(Pk), method='L-BFGS-B', options={'maxiter': 10000})

    print("Optimal solution:\n", res.x)
    print("Objective value:", rosen_b(res.x))
    print("Constraint value:", g(res.x,r))
    print("Number of iterations:", res.nit)
    
    x0 = res.x    # Set xk+1 as new solution
    Pk *= 10      # Pk+1 = Pk*10

    print("---------------------------------------------------------------")    

---------------------------------------------------------------
Testing Penalty Method with Pk = 1
Optimal solution:
 [1.00000217 1.00000007 0.99999929 0.99999863 1.00000213 1.00000216
 1.000002   1.00000158 0.99999679 0.99999517]
Objective value: 1.0195622143936843e-07
Constraint value: -1.1371737329568532e-08
Number of iterations: 70
---------------------------------------------------------------
---------------------------------------------------------------
Testing Penalty Method with Pk = 10
Optimal solution:
 [1.00000217 1.00000007 0.99999929 0.99999863 1.00000213 1.00000216
 1.000002   1.00000158 0.99999679 0.99999517]
Objective value: 1.0195911085323836e-07
Constraint value: -8.869506018527318e-09
Number of iterations: 2
---------------------------------------------------------------
---------------------------------------------------------------
Testing Penalty Method with Pk = 100
Optimal solution:
 [1.00000217 1.00000007 0.99999929 0.99999863 1.00000213 1.00000216
 1.000002 

Now that b = 1000, convergence takes longer.

In previous questions, we talked about how increasing b makes the rosenbrock function more steep, which generally makes optimization more difficult. So multiplying b by a factor of 100 is probably the reason why we have slower convergence.

#### iv)

In [19]:
# (iv)

Pk = 1
b = 10 # We chose to set b back to 10 for this part
r = 1
x0 = np.zeros(n)
for i in range(13):
    print("---------------------------------------------------------------")
    print(f"Testing Penalty Method with Pk = {Pk}")
    res = minimize(penalty_method, x0, args=(Pk), method='L-BFGS-B', options={'maxiter': 10000})

    print("Optimal solution:\n", res.x)
    print("Objective value:", rosen_b(res.x))
    print("Constraint value:", g(res.x,r))
    print("Number of iterations:", res.nit)
    
    x0 = res.x    # Set xk+1 as new solution
    Pk *= 10      # Pk+1 = Pk*10

    print("---------------------------------------------------------------")    

---------------------------------------------------------------
Testing Penalty Method with Pk = 1
Optimal solution:
 [0.96107433 0.95341207 0.95016342 0.94824843 0.94584292 0.94065447
 0.92775836 0.89494054 0.81134143 0.5984372 ]
Objective value: 0.8279437620874046
Constraint value: 7.092149843537962
Number of iterations: 15
---------------------------------------------------------------
---------------------------------------------------------------
Testing Penalty Method with Pk = 10
Optimal solution:
 [0.60316994 0.53485877 0.51114339 0.50323287 0.50007739 0.49703284
 0.48950058 0.46582105 0.38642832 0.07466313]
Objective value: 26.89529967455445
Constraint value: 1.2730192853909386
Number of iterations: 10
---------------------------------------------------------------
---------------------------------------------------------------
Testing Penalty Method with Pk = 100
Optimal solution:
 [0.39795164 0.35177078 0.33780941 0.33315482 0.3312991  0.32950774
 0.3250781  0.3111596  0.264

Now that r = 1 (and b is back to 10), we see that the function does not converge to a matrix of ones.

We do believe the function is converging correctly. But a convergence to a different value suggests that the global minimum does not occur within the specified constraint.

We think that this function has found the local minimum of the rosenbrock that is constrained by a unit sphere of radius r..

### 1(c) Augmented Lagrangian

### 1(d) Parameter Transformation

In [20]:
# define function hi(z)
a = 10
b = 10
def h (z,i,n):
    if i<= n-1:
        num = np.exp(z[i])
        den = 1 + sum(np.exp(z[s]) for s in range(s-1))        
        return np.sqrt(num/den)    
    else: 
        return np.sqrt(1/den)

# define inverse function hi^-1(x)
def h_inverse (x):
    xi = x[-1]
    xn = x[:-1]
    log_thing = np.log(xi/xn)
    return 2*log_thing

# to solve, substitue inverse function into constraint part of multiplier so it becomes, lagrange = (a(1-x[i])**2 + b(x[i+1]-x[i]**2)**2) + lamda (h_inverse)

x0 = np.zeros(n)
res = minimize(rosen_b, x0, method='BFGS') 

print("Optimal solution:\n", res.x)
print("Objective value:", rosen_b(res.x))
print("Constraint value:", g(res.x,r))
print("Number of iterations:", res.nit)


Optimal solution:
 [0.99999999 0.99999999 0.99999998 0.99999999 0.99999996 0.99999998
 0.99999998 0.99999997 0.99999995 0.99999988]
Objective value: 1.1262388848283833e-13
Constraint value: 8.999999326045133
Number of iterations: 24


This method works well.

It is much better than our approach in part a) since it correctly converges and is reliable.

Compared to our approach in b):
- It seems to be more reliable. It does not require us to choose a penalty value, which may or may not break break the function.
- It is faster. It took 24 iterations, while our most ideal scenario in part b) took 33 iterations.
- It is as accurate as our most ideal scenario in part b), but of course this is subject to our function in part b working properly.

Overall, parameter transformation seems to be the best way of finding the global mimimum of our rosenbrock function.